In [1]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA


c:\Users\dhana\Desktop\GenAI Projects\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_parquet("raw_dataset/test_all-00000-of-00001.parquet")

In [3]:
print(df.columns.tolist())


['id', 'district', 'text', 'label']


In [4]:
df = df.drop(columns=['district'])

In [5]:
df.head()

,id,text,label
0,Bail Application_473_202129-01-20213091,{'facts-and-arguments': ['जमानत प्रार्थनापत्र ...,1
1,Bail Application_2568_202030-09-20202372,{'facts-and-arguments': ['समर्थन में राजवीर <न...,1
2,Bail Application_4485_201927-08-2019132,{'facts-and-arguments': ['जमानत प्रार्थना पत्र...,1
3,Bail Application_1603_202105-03-20211467,{'facts-and-arguments': ['समर्थन में सितारा वे...,1
4,Bail Application_2710_202107-04-2021161,{'facts-and-arguments': ['इस जमानत प्रार्थना प...,1


In [6]:
translator_model = "Helsinki-NLP/opus-mt-hi-en"
tokenizer = MarianTokenizer.from_pretrained(translator_model)
model = MarianMTModel.from_pretrained(translator_model)

def translate_hi_to_en(text):
    batch = tokenizer.prepare_seq2seq_batch([text], return_tensors="pt", truncation=True)
    translated = model.generate(**batch)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

df["text"] = df["text"].apply(translate_hi_to_en)

c:\Users\dhana\Desktop\GenAI Projects\llm\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dhana\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-hi-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\dhana\Desktop\GenAI Projects\llm\lib\site-packages\transformers\models\marian\tokenization_

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).